In [5]:
document = {
    'classification': 'Finance',
    'lines': [
        {
            'score': 0.85,
            'tokens': [
                {'text': 'This', 'score': 0.20},
                {'text': 'is', 'score': 0.30},
                {'text': 'a', 'score': 0.40},
                {'text': 'test', 'score': 0.60},
                {'text': 'example', 'score': 0.70}
            ]
        },
        {
            'score': 0.25,
            'tokens': [
                {'text': 'Can', 'score': 0.90},
                {'text': 'it', 'score': 0.60},
                {'text': 'display', 'score': 0.30},
                {'text': 'well?', 'score': 0.20}
            ]
        }
    ]
}

In [6]:
%%javascript
require.config({ 
     paths: { 
     d3: 'https://d3js.org/d3.v6.min'
}});

<IPython.core.display.Javascript object>

In [ ]:
! cp ../ecco/d3.v6.min.js ./

In [7]:
from IPython.display import Javascript, HTML
import json

In [12]:
Javascript("""
 function dispHAN(element, doc) {
     require(['d3'], function(d3) {   
         for (const line of doc['lines']){
             let thisdiv = d3.select(element.get(0)).append('div')
            thisdiv.append('div')
             .style('height', '20px')
             .style('width', '20px')
             .style('background-color', d3.interpolateReds(0.6 * line['score']))
             .style('display', 'inline-block')
             .style('margin-right', '4px')
             .style('vertical-align', 'middle')
             for (const token of line['tokens']){
                 thisdiv
                     .append('span')
                     .text(token['text'] + ' ')
                     .style('background-color', d3.interpolateBlues(0.6*token['score']))
                     .style('margin', '2px')
                     .style('padding', '2px')
            }
         }
     })
 };
 dispHAN(element, %s);
 """ % json.dumps(document))

<IPython.core.display.Javascript object>